# Using CSBM generated data to train a model

In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import math
from MultiClassCSBM import MultiClassCSBM

In [2]:
n = 500
d = 100
c = 20

In [3]:
csbm = MultiClassCSBM(n=n, dimensions=d, classes=c)
data_list = []
for _ in range(10):
    data_list.append(csbm.data)
    csbm.evolve()
print(data_list)

[Data(x=[500, 100], edge_index=[2, 29976], y=[500], train_mask=[500], test_mask=[500]), Data(x=[1000, 100], edge_index=[2, 90013], y=[1000], train_mask=[1000], test_mask=[1000]), Data(x=[1500, 100], edge_index=[2, 180075], y=[1500], train_mask=[1500], test_mask=[1500]), Data(x=[2000, 100], edge_index=[2, 300045], y=[2000], train_mask=[2000], test_mask=[2000]), Data(x=[2500, 100], edge_index=[2, 449745], y=[2500], train_mask=[2500], test_mask=[2500]), Data(x=[3000, 100], edge_index=[2, 630570], y=[3000], train_mask=[3000], test_mask=[3000]), Data(x=[3500, 100], edge_index=[2, 841452], y=[3500], train_mask=[3500], test_mask=[3500]), Data(x=[4000, 100], edge_index=[2, 1081635], y=[4000], train_mask=[4000], test_mask=[4000]), Data(x=[4500, 100], edge_index=[2, 1352746], y=[4500], train_mask=[4500], test_mask=[4500]), Data(x=[5000, 100], edge_index=[2, 1653140], y=[5000], train_mask=[5000], test_mask=[5000])]


In [16]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

dataset = DataLoader(data_list, batch_size=32)
print(len(dataset))

1


In [10]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

AttributeError: 'DataLoader' object has no attribute 'num_node_features'